# 2. Agentes con Function Calling Nativo de OpenAI

## Objetivos de Aprendizaje
- Comprender el mecanismo de "Function Calling" de OpenAI y sus ventajas.
- Definir herramientas en el formato JSON Schema que requiere la API.
- Implementar un agente que utiliza function calling para interactuar con una API externa (Wikipedia).
- Manejar el flujo de una conversación donde el modelo solicita la ejecución de una función.

## ¿Qué es Function Calling y por qué es mejor?

En el notebook anterior, construimos un agente que funcionaba parseando texto. El LLM escribía su intención de usar una herramienta en un formato específico ("Action: {...}"), y nosotros usábamos expresiones regulares para extraer esa intención. Este método funciona, pero es frágil:

- El LLM puede cometer errores y no generar el texto en el formato exacto.
- El parsing puede fallar si la estructura del texto cambia ligeramente.
- Los argumentos de la función se pasan como un string que debemos convertir a JSON, lo cual puede dar errores.

**Function Calling** es la solución nativa de OpenAI a este problema. En lugar de pedirle al modelo que *escriba* qué herramienta quiere usar, le permitimos que nos devuelva una estructura de datos **JSON bien formada** que especifica el nombre de la función y los argumentos que quiere usar. 

**Ventajas:**
1.  **Fiabilidad**: El modelo está entrenado para generar un JSON válido, eliminando casi por completo los errores de formato.
2.  **Seguridad**: Evita la necesidad de ejecutar código que el LLM genera directamente.
3.  **Simplicidad**: No más parsing con expresiones regulares. La intención del modelo es clara y estructurada.

### 1. Instalación y Configuración

In [ ]:
!pip install openai wikipedia -q

In [1]:
import os
import json
import wikipedia
from openai import OpenAI

# Configurar el idioma de Wikipedia
wikipedia.set_lang("es")

# --- Configuración del Cliente OpenAI ---
try:
    client = OpenAI(
        base_url=os.environ.get("GITHUB_BASE_URL"),
        api_key=os.environ.get("GITHUB_TOKEN")
    )
    print("✅ Cliente OpenAI configurado correctamente.")
except Exception as e:
    print(f"❌ Error configurando el cliente: {e}")
    client = None

✅ Cliente OpenAI configurado correctamente.


### 2. Definición de Herramientas en formato OpenAI

Primero, definimos la función de Python que queremos que nuestro agente pueda usar. En este caso, una función que busca un resumen en Wikipedia.

Luego, y esto es lo más importante, describimos esa función en un formato de **JSON Schema**. Esta descripción le dice al LLM qué es la herramienta, para qué sirve y qué argumentos necesita.

In [2]:
# La función de Python que realiza la acción
def get_wikipedia_summary(query):
    """Busca en Wikipedia un tema y devuelve un resumen de 2 frases."""
    try:
        # sentences=2 pide a la librería que devuelva un resumen de 2 frases
        summary = wikipedia.summary(query, sentences=2)
        return summary
    except wikipedia.exceptions.PageError:
        return f"No se encontró ninguna página para '{query}'."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"La búsqueda para '{query}' es ambigua. Opciones: {e.options[:3]}"

# Lista de herramientas en formato JSON Schema para la API de OpenAI
tools_definition = [
    {
        "type": "function",
        "function": {
            "name": "get_wikipedia_summary",
            "description": "Obtiene un resumen conciso de un artículo de Wikipedia para un tema o persona específica.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "El tema o nombre a buscar en Wikipedia. Por ejemplo, 'Albert Einstein'."
                    }
                },
                "required": ["query"]
            }
        }
    }
]

print("✅ Herramienta y su definición para OpenAI listas.")

✅ Herramienta y su definición para OpenAI listas.


### 3. El Flujo del Agente con Function Calling

El proceso ahora es más estructurado:

1.  **Usuario -> Agente**: Enviamos la pregunta del usuario y la lista de herramientas (`tools_definition`) al LLM.
2.  **LLM -> Agente**: El LLM analiza la pregunta. Si decide que necesita una herramienta, en lugar de devolver un mensaje de texto, devuelve un objeto `tool_calls`.
3.  **Agente**: Verificamos si la respuesta contiene `tool_calls`. Si es así, ejecutamos la función correspondiente en nuestro código Python.
4.  **Agente -> LLM**: Enviamos el resultado de la función de vuelta al LLM en un nuevo mensaje con `role="tool"`.
5.  **LLM -> Usuario**: El LLM, ahora con la información de la herramienta, genera la respuesta final en lenguaje natural.

In [4]:
def run_agent_with_function_calling(user_query, client, tools_definition):
    if not client:
        return "Cliente no inicializado."

    # Mapeo de nombres de función a las funciones de Python reales
    available_tools = {
        "get_wikipedia_summary": get_wikipedia_summary,
    }

    messages = [{"role": "user", "content": user_query}]
    
    print(f"--- 🚀 Iniciando agente para la consulta: '{user_query}' ---")

    # Primera llamada al modelo
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools_definition, # Aquí pasamos la definición de las herramientas
        tool_choice="auto",  # El modelo decide si usar una herramienta o no
    )

    response_message = response.choices[0].message
    messages.append(response_message) # Añadir la respuesta del LLM al historial

    # Comprobar si el modelo quiere llamar a una función
    if response_message.tool_calls:
        print("🤖 El modelo ha decidido usar una herramienta...")
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"   - Herramienta: {function_name}")
            print(f"   - Argumentos: {function_args}")
            
            # Ejecutar la función
            function_to_call = available_tools[function_name]
            function_response = function_to_call(**function_args)
            
            print(f"   - Resultado: {function_response}")
            
            # Enviar el resultado de vuelta al modelo
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        
        # Segunda llamada al modelo, ahora con el resultado de la herramienta
        print("🧠 El modelo está procesando el resultado de la herramienta...")
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        return second_response.choices[0].message.content
    else:
        # Si el modelo no usó una herramienta, devuelve su respuesta directamente
        print("✅ El modelo ha respondido directamente.")
        return response_message.content

print("✅ Lógica del agente con Function Calling definida.")

✅ Lógica del agente con Function Calling definida.


### 4. Ejecución del Agente

Probemos con una pregunta que claramente necesita conocimiento externo.

In [5]:
query = "¿Quién fue Marie Curie y cuáles fueron sus logros más importantes?"
final_answer = run_agent_with_function_calling(query, client, tools_definition)

print(f"🏁 Respuesta Final del Agente:{final_answer}")

--- 🚀 Iniciando agente para la consulta: '¿Quién fue Marie Curie y cuáles fueron sus logros más importantes?' ---
🤖 El modelo ha decidido usar una herramienta...
   - Herramienta: get_wikipedia_summary
   - Argumentos: {'query': 'Marie Curie'}
   - Resultado: Maria Salomea Skłodowska-Curie,[A]​[B]​ más conocida como Marie Curie[C]​[B]​ o Madame Curie (Varsovia, 7 de noviembre de 1867-Passy, 4 de julio de 1934), fue una física y química de origen polaco. Pionera en el campo de la radiactividad, es la primera y única persona en recibir dos premios Nobel en distintas especialidades científicas: Física y Química.[D]​ También fue la primera mujer en ocupar el puesto de profesora en la Universidad de París y la primera en recibir sepultura con honores en el Panteón de París por méritos propios en 1995.[E]​
Nació en Varsovia, en lo que entonces era el Zarato de Polonia (territorio administrado por el Imperio ruso).
🧠 El modelo está procesando el resultado de la herramienta...
🏁 Respuesta Fina

## Conclusiones

El uso de **Function Calling** nativo representa un salto cualitativo en la construcción de agentes. La comunicación entre el LLM y nuestro código es ahora mucho más **robusta, predecible y fácil de depurar**.

Hemos eliminado la necesidad de crear prompts complejos para el formato ReAct y de parsear la salida del modelo. Sin embargo, todavía gestionamos manualmente el estado de la conversación (`messages`) y el flujo de llamadas.

En el próximo notebook, introduciremos **LangChain**, un framework que abstrae esta lógica y nos permite construir agentes aún más potentes con mucho menos código.